# Обучение модели CRNN для распознавания капчи

Этот notebook предназначен для обучения модели на Google Colab.

## Шаги:
1. Установка зависимостей
2. Загрузка файлов проекта
3. Генерация/загрузка датасета
4. Обучение модели
5. Сохранение результатов


In [4]:
# Установка зависимостей
%pip install captcha>=0.7.1 tqdm Pillow torch torchvision matplotlib google


## Загрузка файлов проекта

Вы можете загрузить файлы проекта несколькими способами:

### Вариант 1: Загрузить с GitHub (если проект в репозитории)
### Вариант 2: Загрузить файлы вручную через файловую систему Colab
### Вариант 3: Создать файлы прямо в Colab (клетки ниже)


In [2]:
# Создаем необходимые файлы проекта
# Если файлы уже загружены, эту ячейку можно пропустить

import os
os.makedirs('project', exist_ok=True)

# Если у вас есть файлы в Google Drive, подключите его (НЕОБЯЗАТЕЛЬНО):
# Эта ячейка опциональна - файлы будут созданы в следующих ячейках

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    
    # Копируем файлы, если они есть
    if os.path.exists('/content/drive/MyDrive/captcha_ii'):
        import shutil
        src = '/content/drive/MyDrive/captcha_ii'
        dst = './project'
        try:
            for item in os.listdir(src):
                s = os.path.join(src, item)
                d = os.path.join(dst, item)
                if os.path.isdir(s):
                    shutil.copytree(s, d, dirs_exist_ok=True)
                else:
                    shutil.copy2(s, d)
            print("✓ Файлы загружены из Google Drive")
        except Exception as copy_error:
            print(f"ℹ Файлы не найдены в Drive или ошибка копирования: {copy_error}")
            print("ℹ Файлы будут созданы в следующих ячейках")
    else:
        print("ℹ Папка captcha_ii не найдена в Drive. Файлы будут созданы в следующих ячейках.")
except Exception as e:
    print(f"ℹ Не удалось подключить Google Drive: {e}")
    print("ℹ Это нормально - файлы будут созданы в следующих ячейках")

print("Готово к созданию/загрузке файлов проекта")


ℹ Не удалось подключить Google Drive: mount failed
ℹ Это нормально - файлы будут созданы в следующих ячейках
Готово к созданию/загрузке файлов проекта


In [6]:
# Загружаем файлы model.py, dataset_loader.py, train.py
# Если файлы загружены через Drive или GitHub, раскомментируйте соответствующие строки

# Для загрузки с GitHub:
!git clone https://github.com/alexeii89/captcha_ii.git
%cd captcha_ii

# Или загрузите файлы через интерфейс Colab (Files -> Upload)
# Затем скопируйте в рабочую директорию:
!cp model.py dataset_loader.py train.py main.py ./

print("Файлы должны быть в текущей директории")
!ls -la *.py 2>/dev/null || echo "Файлы .py не найдены. Загрузите их через интерфейс Colab или используйте следующий вариант."


Cloning into 'captcha_ii'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 17.67 KiB | 8.83 MiB/s, done.
/content/captcha_ii
cp: 'model.py' and './model.py' are the same file
cp: 'dataset_loader.py' and './dataset_loader.py' are the same file
cp: 'train.py' and './train.py' are the same file
cp: 'main.py' and './main.py' are the same file
Файлы должны быть в текущей директории
-rw-r--r-- 1 root root  7548 Dec 10 06:10 dataset_loader.py
-rw-r--r-- 1 root root  3190 Dec 10 06:10 main.py
-rw-r--r-- 1 root root  5126 Dec 10 06:10 model.py
-rw-r--r-- 1 root root  3521 Dec 10 06:10 predict.py
-rw-r--r-- 1 root root 10180 Dec 10 06:10 train.py


## Вариант: Создать файлы прямо здесь

Если файлы не загружены, можно прочитать их из локальных файлов и создать в Colab.


In [ ]:
# Создаем файл model.py
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F


class CRNN(nn.Module):
    """
    CRNN (Convolutional Recurrent Neural Network) модель для распознавания капчи.
    Использует CNN для извлечения признаков и RNN (LSTM) для обработки последовательности.
    """
    
    def __init__(self, num_classes, img_height=80, img_width=200):
        super(CRNN, self).__init__()
        self.num_classes = num_classes
        self.img_height = img_height
        self.img_width = img_width
        
        # CNN для извлечения признаков
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 1)),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 1)),
            
            nn.Conv2d(512, 512, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )
        
        self.rnn_input_size = 512
        self.hidden_size = 256
        self.num_layers = 2
        
        self.rnn = nn.LSTM(
            self.rnn_input_size,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            bidirectional=True
        )
        
        self.fc = nn.Linear(self.hidden_size * 2, num_classes)
        
    def forward(self, x):
        conv_features = self.cnn(x)
        batch_size, channels, height, width = conv_features.size()
        
        if height > 1:
            conv_features = conv_features.mean(dim=2)
        else:
            conv_features = conv_features.squeeze(2)
        
        conv_features = conv_features.permute(0, 2, 1)
        rnn_out, _ = self.rnn(conv_features)
        output = self.fc(rnn_out)
        output = output.log_softmax(2)
        
        return output


def decode_predictions(predictions, dataset):
    pred_indices = predictions.argmax(dim=2)
    decoded_texts = []
    for pred_seq in pred_indices:
        text = dataset.decode(pred_seq)
        decoded_texts.append(text)
    return decoded_texts


In [ ]:
# Создаем файл dataset_loader.py
%%writefile dataset_loader.py
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import platform


class CaptchaDataset(Dataset):
    def __init__(self, dataset_dir, characters, transform=None):
        self.dataset_dir = dataset_dir
        self.characters = characters
        self.transform = transform
        
        self.char_to_idx = {char: idx + 1 for idx, char in enumerate(characters)}
        self.idx_to_char = {idx: char for char, idx in self.char_to_idx.items()}
        self.idx_to_char[0] = ''
        self.num_classes = len(characters) + 1
        
        self.image_files = [f for f in os.listdir(dataset_dir) if f.endswith('.png')]
        print(f"Загружено {len(self.image_files)} изображений")
        print(f"Количество символов: {len(characters)}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        filename = self.image_files[idx]
        filepath = os.path.join(self.dataset_dir, filename)
        image = Image.open(filepath).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        text = filename.replace('.png', '').rsplit('_', 1)[0]
        target = [self.char_to_idx[char] for char in text]
        target_length = len(target)
        
        return image, torch.tensor(target, dtype=torch.long), target_length, text
    
    def decode(self, indices):
        if isinstance(indices, torch.Tensor):
            indices = indices.cpu().numpy()
        decoded = []
        prev_idx = None
        for idx in indices:
            if idx != 0 and idx != prev_idx and idx < len(self.char_to_idx) + 1:
                char = self.idx_to_char.get(idx, '')
                if char:
                    decoded.append(char)
            prev_idx = idx
        return ''.join(decoded)


class DatasetWrapper(torch.utils.data.Dataset):
    def __init__(self, subset, original_dataset):
        self.subset = subset
        self.dataset = original_dataset
    
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        return self.subset[idx]
    
    @property
    def char_to_idx(self):
        return self.dataset.char_to_idx
    
    @property
    def idx_to_char(self):
        return self.dataset.idx_to_char
    
    @property
    def num_classes(self):
        return self.dataset.num_classes
    
    def decode(self, indices):
        return self.dataset.decode(indices)


def get_data_loaders(dataset_dir, characters, batch_size=32, train_split=0.8, num_workers=None):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    full_dataset = CaptchaDataset(dataset_dir, characters, transform=transform)
    
    train_size = int(train_split * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_dataset = DatasetWrapper(train_subset, full_dataset)
    val_dataset = DatasetWrapper(val_subset, full_dataset)
    
    if num_workers is None:
        num_workers = 2  # Для Colab используем 2 workers
    
    use_pin_memory = torch.cuda.is_available()
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, collate_fn=collate_fn, pin_memory=use_pin_memory
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, collate_fn=collate_fn, pin_memory=use_pin_memory
    )
    
    return train_loader, val_loader, full_dataset


def collate_fn(batch):
    images, targets, target_lengths, texts = zip(*batch)
    images = torch.stack(images, 0)
    max_len = max(target_lengths)
    padded_targets = torch.zeros(len(targets), max_len, dtype=torch.long)
    for i, (target, length) in enumerate(zip(targets, target_lengths)):
        padded_targets[i, :length] = target
    target_lengths = torch.tensor(target_lengths, dtype=torch.long)
    return images, padded_targets, target_lengths, texts


In [ ]:
# Создаем файл main.py для генерации датасета
%%writefile main.py
import os
import random
from captcha.image import ImageCaptcha
from tqdm import tqdm


def generate_captcha_dataset(num_images=50000):
    """
    Генерирует датасет из изображений капчи с русскими буквами и цифрами.
    Имя файла соответствует тексту на капче.
    
    Args:
        num_images: Количество изображений для генерации (по умолчанию 50000)
    """
    dataset_dir = 'dataset'
    os.makedirs(dataset_dir, exist_ok=True)
    
    russian_letters = 'АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
    numbers = '0123456789'
    characters = russian_letters + numbers
    
    min_length = 4
    max_length = 6
    
    image_captcha = ImageCaptcha(width=200, height=80)
    created_files = set()
    
    print(f"Начинаю генерацию {num_images} изображений капчи...")
    print(f"Символы: {characters}")
    print(f"Длина текста: {min_length}-{max_length} символов")
    print(f"Папка для сохранения: {dataset_dir}")
    
    for i in tqdm(range(num_images), desc="Генерация капчи"):
        length = random.randint(min_length, max_length)
        captcha_text = ''.join(random.choices(characters, k=length))
        filename = f"{captcha_text}.png"
        filepath = os.path.join(dataset_dir, filename)
        
        counter = 1
        while filename in created_files:
            filename = f"{captcha_text}_{counter}.png"
            filepath = os.path.join(dataset_dir, filename)
            counter += 1
        
        created_files.add(filename)
        image = image_captcha.generate_image(captcha_text)
        image.save(filepath)
    
    print(f"\n✓ Готово! Сгенерировано {num_images} изображений в папке '{dataset_dir}'")
    print(f"✓ Уникальных файлов: {len(created_files)}")


if __name__ == "__main__":
    generate_captcha_dataset()


## Генерация датасета

Теперь сгенерируем датасет. Вы можете:
1. Сгенерировать на Colab (займет время)
2. Загрузить готовый датасет с Google Drive


In [8]:
# Вариант 1: Генерация датасета прямо в Colab
# Раскомментируйте следующие строки, если хотите сгенерировать датасет

from main import generate_captcha_dataset

# Генерируем 50000 изображений (можете уменьшить для теста)
generate_captcha_dataset(num_images=50000)


TypeError: generate_captcha_dataset() got an unexpected keyword argument 'num_images'

In [ ]:
# Вариант 2: Загрузка датасета с Google Drive (ОПЦИОНАЛЬНО)
# Раскомментируйте и запустите, если у вас уже есть датасет в Drive

# try:
#     from google.colab import drive
#     drive.mount('/content/drive', force_remount=False)
#     
#     # Создаем симлинк на папку с датасетом
#     if os.path.exists('/content/drive/MyDrive/captcha_ii/dataset'):
#         !ln -s /content/drive/MyDrive/captcha_ii/dataset ./dataset
#         print("✓ Датасет подключен из Google Drive")
#     else:
#         print("ℹ Датасет не найден в Drive, будет сгенерирован новый")
# except Exception as e:
#     print(f"ℹ Не удалось подключить Drive: {e}")
#     print("ℹ Датасет будет сгенерирован в следующей ячейке")


## Обучение модели

Теперь запускаем обучение. Убедитесь, что датасет готов.


In [ ]:
# Импортируем необходимые модули
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from model import CRNN, decode_predictions
from dataset_loader import get_data_loaders
import matplotlib.pyplot as plt
import os


In [ ]:
# Функции для обучения (можно вынести в train.py, но для простоты оставим здесь)

def train_epoch(model, train_loader, criterion, optimizer, device, dataset):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc="Обучение")
    for images, targets, target_lengths, texts in progress_bar:
        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        outputs = outputs.permute(1, 0, 2)
        
        input_lengths = torch.full(
            size=(outputs.size(1),),
            fill_value=outputs.size(0),
            dtype=torch.long
        ).to(device)
        
        loss = criterion(outputs, targets, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        with torch.no_grad():
            pred_outputs = model(images)
            decoded_preds = decode_predictions(pred_outputs, dataset)
            
            for pred, true in zip(decoded_preds, texts):
                if pred == true:
                    correct += 1
                total += 1
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })
    
    return total_loss / len(train_loader), 100 * correct / total if total > 0 else 0


def validate(model, val_loader, criterion, device, dataset):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Валидация")
        for images, targets, target_lengths, texts in progress_bar:
            images = images.to(device)
            targets = targets.to(device)
            target_lengths = target_lengths.to(device)
            
            outputs = model(images)
            outputs = outputs.permute(1, 0, 2)
            
            input_lengths = torch.full(
                size=(outputs.size(1),),
                fill_value=outputs.size(0),
                dtype=torch.long
            ).to(device)
            
            loss = criterion(outputs, targets, input_lengths, target_lengths)
            total_loss += loss.item()
            
            pred_outputs = model(images)
            decoded_preds = decode_predictions(pred_outputs, dataset)
            
            for pred, true in zip(decoded_preds, texts):
                if pred == true:
                    correct += 1
                total += 1
            
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * correct / total:.2f}%'
            })
    
    return total_loss / len(val_loader), 100 * correct / total if total > 0 else 0


In [ ]:
# Настройки обучения
russian_letters = 'АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
numbers = '0123456789'
characters = russian_letters + numbers

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DATASET_DIR = 'dataset'

print("=" * 60)
print("Настройки обучения")
print("=" * 60)
print(f"Эпох: {EPOCHS}")
print(f"Размер батча: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Количество символов: {len(characters)}")
print("=" * 60)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# Загружаем данные
print("\nЗагрузка датасета...")
train_loader, val_loader, dataset = get_data_loaders(
    dataset_dir=DATASET_DIR,
    characters=characters,
    batch_size=BATCH_SIZE,
    train_split=0.8,
    num_workers=2  # Для Colab
)

print(f"Обучающих примеров: {len(train_loader.dataset)}")
print(f"Валидационных примеров: {len(val_loader.dataset)}")

# Создаем модель
model = CRNN(num_classes=len(characters) + 1).to(device)
print(f"\nМодель создана. Параметров: {sum(p.numel() for p in model.parameters()):,}")

# Функция потерь и оптимизатор
criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# История
train_losses, train_accs = [], []
val_losses, val_accs = [], []

os.makedirs('models', exist_ok=True)
best_val_acc = 0

print("\nНачинаем обучение...\n")


In [ ]:
# Цикл обучения
for epoch in range(EPOCHS):
    print(f"\nЭпоха {epoch + 1}/{EPOCHS}")
    print("-" * 60)
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    val_loss, val_acc = validate(model, val_loader, criterion, device, dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    
    print(f"\nРезультаты эпохи {epoch + 1}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    if old_lr != new_lr:
        print(f"  Learning rate: {old_lr:.6f} -> {new_lr:.6f}")
    
    # Сохраняем лучшую модель
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'characters': characters,
        }, 'models/best_model.pth')
        print(f"  ✓ Сохранена лучшая модель (Val Acc: {val_acc:.2f}%)")
    
    # Чекпоинт каждые 10 эпох
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs,
            'characters': characters,
        }, f'models/checkpoint_epoch_{epoch + 1}.pth')

# Финальное сохранение
torch.save({
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accs': train_accs,
    'val_accs': val_accs,
    'characters': characters,
}, 'models/final_model.pth')

print("\n" + "=" * 60)
print("Обучение завершено!")
print(f"Лучшая точность на валидации: {best_val_acc:.2f}%")
print("=" * 60)


In [ ]:
# Строим графики
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses, label='Train Loss')
ax1.plot(val_losses, label='Val Loss')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Loss')
ax1.set_title('История Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(train_accs, label='Train Acc')
ax2.plot(val_accs, label='Val Acc')
ax2.set_xlabel('Эпоха')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('История Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('models/training_history.png')
plt.show()

print("График сохранен: models/training_history.png")


## Сохранение результатов

Сохраните модель в Google Drive для дальнейшего использования.


In [ ]:
# Сохранение в Google Drive (ОПЦИОНАЛЬНО)
# Запустите эту ячейку, чтобы сохранить модели в Drive

try:
    from google.colab import drive
    import shutil
    
    drive.mount('/content/drive', force_remount=False)
    
    # Создаем папку, если её нет
    drive_path = '/content/drive/MyDrive/captcha_ii'
    os.makedirs(drive_path, exist_ok=True)
    
    # Копируем модели
    models_src = './models'
    models_dst = os.path.join(drive_path, 'models')
    
    if os.path.exists(models_src):
        if os.path.exists(models_dst):
            shutil.rmtree(models_dst)
        shutil.copytree(models_src, models_dst)
        print("✓ Модели сохранены в Google Drive:")
        print(f"  /content/drive/MyDrive/captcha_ii/models/")
    else:
        print("ℹ Папка models не найдена. Сначала запустите обучение.")
except Exception as e:
    print(f"ℹ Не удалось сохранить в Drive: {e}")
    print("\nℹ Используйте скачивание через интерфейс Colab:")
    print("   1. Откройте панель Files (слева)")
    print("   2. Перейдите в папку models/")
    print("   3. Нажмите правой кнопкой на best_model.pth")
    print("   4. Выберите 'Download'")
